In [1]:
import pandas as pd
import numpy as np
import os
from queue import PriorityQueue

In [2]:
quotes_filename = os.path.join("play_data", "XBTUSD_quotes_191214_0434.csv")
trades_filename = os.path.join("play_data", "XBTUSD_trades_191214_0434.csv")

In [3]:
quotes_full = pd.read_csv(
    quotes_filename,
    index_col='recorded',
    parse_dates=['timestamp', 'recorded']
)

quotes_full

,timestamp,bidSize,bidPrice,askPrice,askSize
recorded,,,,,
2019-12-14 04:34:40.731941+00:00,2019-12-14 04:34:40.024000+00:00,3840427,7251.5,7252.0,701477
2019-12-14 04:34:41.211565+00:00,2019-12-14 04:34:40.410000+00:00,3840391,7251.5,7252.0,701477
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.165000+00:00,3840391,7251.5,7252.0,731477
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.183000+00:00,3840187,7251.5,7252.0,731277
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.431000+00:00,3840187,7251.5,7252.0,715277
...,...,...,...,...,...
2019-12-14 11:21:25.279438+00:00,2019-12-14 11:21:24.810000+00:00,1737201,7171.0,7171.5,2502314
2019-12-14 11:21:25.279438+00:00,2019-12-14 11:21:24.971000+00:00,1737201,7171.0,7171.5,2562314
2019-12-14 11:21:25.279438+00:00,2019-12-14 11:21:24.980000+00:00,1681090,7171.0,7171.5,2562314


In [ ]:
trades_full = pd.read_csv(
    trades_filename,
    index_col='received',
    parse_dates=['timestamp', 'received']
)
trades_full

In [27]:
FULL_TABLES = {}
FULL_TABLES["trades"] = trades_full
FULL_TABLES["quotes"] = quotes_full
TABLES = {}

In [28]:
quotes_updates = pd.DataFrame(
    "quotes",
    index=quotes_full.index.drop_duplicates(),
    columns=["table"]
)

trades_updates = pd.DataFrame(
    "trades",
    index=trades_full.index.drop_duplicates(),
    columns=["table"]
)

trades_updates.head()

,table
received,
2019-12-14 04:34:40.742081+00:00,trades
2019-12-14 04:34:42.499730+00:00,trades
2019-12-14 04:34:48.168746+00:00,trades
2019-12-14 04:34:48.332053+00:00,trades
2019-12-14 04:34:50.665331+00:00,trades


In [32]:
pq = PriorityQueue()

In [40]:
for index, values in trades_updates.head(20).iterrows():
    event = {}
    event["type"] = "data"
    event["table"] = values["table"]
    pq.put((index, event))
    
for index, values in quotes_updates.head(20).iterrows():
    event = {}
    event["type"] = "data"
    event["table"] = values["table"]
    pq.put((index, event))
    
while not pq.empty():
    curr_time, event = pq.get()
    if event["type"] == "data":
        table = event["table"]
        TABLES[table] = FULL_TABLES[table].loc[:curr_time]
        
    for i, j in TABLES.items():
        display(j)

,timestamp,side,size,price
received,,,,
2019-12-14 04:34:40.742081+00:00,2019-12-14 04:34:38.713000+00:00,Sell,2500,7251.5
2019-12-14 04:34:42.499730+00:00,2019-12-14 04:34:42.378000+00:00,Buy,12000,7252.0
2019-12-14 04:34:48.168746+00:00,2019-12-14 04:34:48.044000+00:00,Buy,100000,7252.0
2019-12-14 04:34:48.332053+00:00,2019-12-14 04:34:48.213000+00:00,Buy,116,7252.0
2019-12-14 04:34:50.665331+00:00,2019-12-14 04:34:50.504000+00:00,Buy,2500,7252.0
...,...,...,...,...
2019-12-14 04:37:23.640673+00:00,2019-12-14 04:37:23.521000+00:00,Buy,5000,7252.0
2019-12-14 04:37:24.695565+00:00,2019-12-14 04:37:24.575000+00:00,Sell,52,7251.5
2019-12-14 04:37:26.154660+00:00,2019-12-14 04:37:26.031000+00:00,Sell,1,7251.5


,timestamp,bidSize,bidPrice,askPrice,askSize
recorded,,,,,
2019-12-14 04:34:40.731941+00:00,2019-12-14 04:34:40.024000+00:00,3840427,7251.5,7252.0,701477


,timestamp,side,size,price
received,,,,
2019-12-14 04:34:40.742081+00:00,2019-12-14 04:34:38.713000+00:00,Sell,2500,7251.5


,timestamp,bidSize,bidPrice,askPrice,askSize
recorded,,,,,
2019-12-14 04:34:40.731941+00:00,2019-12-14 04:34:40.024000+00:00,3840427,7251.5,7252.0,701477


,timestamp,side,size,price
received,,,,
2019-12-14 04:34:40.742081+00:00,2019-12-14 04:34:38.713000+00:00,Sell,2500,7251.5


...

,timestamp,bidSize,bidPrice,askPrice,askSize
recorded,,,,,
2019-12-14 04:34:40.731941+00:00,2019-12-14 04:34:40.024000+00:00,3840427,7251.5,7252.0,701477
2019-12-14 04:34:41.211565+00:00,2019-12-14 04:34:40.410000+00:00,3840391,7251.5,7252.0,701477
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.165000+00:00,3840391,7251.5,7252.0,731477
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.183000+00:00,3840187,7251.5,7252.0,731277
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.431000+00:00,3840187,7251.5,7252.0,715277
...,...,...,...,...,...
2019-12-14 04:34:58.210954+00:00,2019-12-14 04:34:57.748000+00:00,3799528,7251.5,7252.0,641095
2019-12-14 04:34:58.210954+00:00,2019-12-14 04:34:57.926000+00:00,3799528,7251.5,7252.0,641395
2019-12-14 04:34:58.210954+00:00,2019-12-14 04:34:57.941000+00:00,3780487,7251.5,7252.0,641395


,timestamp,side,size,price
received,,,,
2019-12-14 04:34:40.742081+00:00,2019-12-14 04:34:38.713000+00:00,Sell,2500,7251.5
2019-12-14 04:34:42.499730+00:00,2019-12-14 04:34:42.378000+00:00,Buy,12000,7252.0
2019-12-14 04:34:48.168746+00:00,2019-12-14 04:34:48.044000+00:00,Buy,100000,7252.0
2019-12-14 04:34:48.332053+00:00,2019-12-14 04:34:48.213000+00:00,Buy,116,7252.0
2019-12-14 04:34:50.665331+00:00,2019-12-14 04:34:50.504000+00:00,Buy,2500,7252.0
2019-12-14 04:34:54.103724+00:00,2019-12-14 04:34:53.984000+00:00,Buy,6000,7252.0
2019-12-14 04:34:54.520173+00:00,2019-12-14 04:34:54.313000+00:00,Buy,100,7252.0
2019-12-14 04:34:56.938605+00:00,2019-12-14 04:34:56.815000+00:00,Sell,688,7251.5
2019-12-14 04:34:57.590202+00:00,2019-12-14 04:34:57.469000+00:00,Buy,1000,7252.0


,timestamp,bidSize,bidPrice,askPrice,askSize
recorded,,,,,
2019-12-14 04:34:40.731941+00:00,2019-12-14 04:34:40.024000+00:00,3840427,7251.5,7252.0,701477
2019-12-14 04:34:41.211565+00:00,2019-12-14 04:34:40.410000+00:00,3840391,7251.5,7252.0,701477
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.165000+00:00,3840391,7251.5,7252.0,731477
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.183000+00:00,3840187,7251.5,7252.0,731277
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.431000+00:00,3840187,7251.5,7252.0,715277
...,...,...,...,...,...
2019-12-14 04:34:58.210954+00:00,2019-12-14 04:34:57.748000+00:00,3799528,7251.5,7252.0,641095
2019-12-14 04:34:58.210954+00:00,2019-12-14 04:34:57.926000+00:00,3799528,7251.5,7252.0,641395
2019-12-14 04:34:58.210954+00:00,2019-12-14 04:34:57.941000+00:00,3780487,7251.5,7252.0,641395


,timestamp,side,size,price
received,,,,
2019-12-14 04:34:40.742081+00:00,2019-12-14 04:34:38.713000+00:00,Sell,2500,7251.5
2019-12-14 04:34:42.499730+00:00,2019-12-14 04:34:42.378000+00:00,Buy,12000,7252.0
2019-12-14 04:34:48.168746+00:00,2019-12-14 04:34:48.044000+00:00,Buy,100000,7252.0
2019-12-14 04:34:48.332053+00:00,2019-12-14 04:34:48.213000+00:00,Buy,116,7252.0
2019-12-14 04:34:50.665331+00:00,2019-12-14 04:34:50.504000+00:00,Buy,2500,7252.0
2019-12-14 04:34:54.103724+00:00,2019-12-14 04:34:53.984000+00:00,Buy,6000,7252.0
2019-12-14 04:34:54.520173+00:00,2019-12-14 04:34:54.313000+00:00,Buy,100,7252.0
2019-12-14 04:34:56.938605+00:00,2019-12-14 04:34:56.815000+00:00,Sell,688,7251.5
2019-12-14 04:34:57.590202+00:00,2019-12-14 04:34:57.469000+00:00,Buy,1000,7252.0


,timestamp,bidSize,bidPrice,askPrice,askSize
recorded,,,,,
2019-12-14 04:34:40.731941+00:00,2019-12-14 04:34:40.024000+00:00,3840427,7251.5,7252.0,701477
2019-12-14 04:34:41.211565+00:00,2019-12-14 04:34:40.410000+00:00,3840391,7251.5,7252.0,701477
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.165000+00:00,3840391,7251.5,7252.0,731477
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.183000+00:00,3840187,7251.5,7252.0,731277
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.431000+00:00,3840187,7251.5,7252.0,715277
...,...,...,...,...,...
2019-12-14 04:34:58.210954+00:00,2019-12-14 04:34:57.748000+00:00,3799528,7251.5,7252.0,641095
2019-12-14 04:34:58.210954+00:00,2019-12-14 04:34:57.926000+00:00,3799528,7251.5,7252.0,641395
2019-12-14 04:34:58.210954+00:00,2019-12-14 04:34:57.941000+00:00,3780487,7251.5,7252.0,641395
